In [ ]:
import google.generativeai as genai
from PIL import Image
import os
import pandas as pd
import torch
!pip install diffusers
from diffusers import StableDiffusionPipeline

In [14]:
api_key = '###'

In [16]:
folder_path = "/content/drive/MyDrive/Myntra2/MYNTRA"

In [ ]:
genai.configure(api_key=api_key)
genModel = genai.GenerativeModel('gemini-pro-vision')

def generate_vintage_prompt(image_name):
    return f"Hey Gemini, describe this vintage cloth '{image_name}' from a designer's perspective."

def generate_trendy_prompt(image_path):
    return f"Hey Gemini, create a trendy and contemporary description inspired by this image."

In [ ]:
# Iterate through images in the folder
data = []
for idx, image_name in enumerate(os.listdir(folder_path)):
    image_path = os.path.join(folder_path, image_name)

    # Generate vintage description prompt
    vintage_prompt = generate_vintage_prompt(image_name)
    vintage_response = genModel.generate_content([vintage_prompt, Image.open(image_path)])
    vintage_description = vintage_response.text if vintage_response.parts else "No vintage description available."

    # Generate trendy description prompt with image only
    trendy_prompt = generate_trendy_prompt(image_path)
    trendy_response = genModel.generate_content([trendy_prompt, Image.open(image_path)])
    trendy_description = trendy_response.text if trendy_response.parts else "No trendy description available."

    data.append({
        'ID': idx,
        'Image': image_name,
        'Vintage Description': vintage_description,
        'Trendy Description': trendy_description
    })

In [ ]:
df = pd.DataFrame(data)
excel_file = '/content/drive/MyDrive/Myntra2/MYNTRA/Data.xlsx'
df.to_excel(excel_file, index=False)

print(f"Descriptions saved to {excel_file}")

In [ ]:
model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
pipe = pipe.to("cuda")

output_folder = "/content/drive/MyDrive/Myntra2/MYNTRA/GeneratedImages"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
for index, row in df.iterrows():
    prompt = row['Trendy Description']
    image = pipe(prompt).images[0]

    generated_image_path = os.path.join(output_folder, row['Image'])
    image.save(generated_image_path)
    print(f"Generated image saved: {generated_image_path}")